# アルゴリズムの検討
- テクニカル分析を交えて売り・買いを判断する

In [ ]:
from pathlib import Path
from itertools import accumulate

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import stock

In [ ]:
# データの準備
sp500_path = Path("../data/us_etf/SPXL.csv")
dow_path = Path("../data/us_etf/DIA.csv")

topix_path = Path("")
nikkei_path = Path("../data/etfs/1458.csv")
nikkei_inv_path = Path("../data/etfs/1459.csv")

sp500_df = pd.read_csv(sp500_path)
dow_df = pd.read_csv(dow_path)
nikkei_df = pd.read_csv(nikkei_path)
nikkei_inv_df = pd.read_csv(nikkei_inv_path)

sp500 = sp500_df.to_numpy()
dow = dow_df.to_numpy()
nikkei = nikkei_df.to_numpy()
nikkei_inv = nikkei_inv_df.to_numpy()

In [ ]:
# データの可視化
start = 0
end = 400
window_size = 7
fig, ax = stock.visualize.boxplot(sp500[start:end, 1], sp500[start:end, 2], sp500[start:end, 3], sp500[start:end, 4])
fig, ax = stock.visualize.moving_average(sp500[start:end, 1], window_size, fig=fig, ax=ax)

## 移動平均に沿って売買してみる
- 移動平均の傾きが負から正になったら買い、正から負になったら売り

In [ ]:
def simple_moving_average(array, window_size=5):
    return np.convolve(array, np.ones(window_size) / window_size, mode="full")[:len(array)]

def weighted_moving_average(array, window_size=5):
    weights = np.arange(1, window_size + 1)
    return np.convolve(array, weights / weights.sum(), mode="full")[:len(array)]


def alg_moving_average(data, window_size=5):
    """
    """
    avg = simple_moving_average(data[:, 1], window_size)
    # avg = weighted_moving_average(data[:, 1], window_size)
    grads = avg[1:] - avg[:-1]

    buys = []
    sells = []
    indices = []
    prev = 0    
    for idx, grad in enumerate(grads):
        if grad > 0 and prev < 0 and len(buys) == len(sells):
            buys.append(data[idx + 1, 1])
        elif grad < 0 and prev > 0 and len(buys) > len(sells):
            sells.append(data[idx + 1, 1])
            indices.append(idx)
        # if grad > 0 and prev < 0 and len(buys) < len(sells):
        #     buys.append(data[idx + 1, 1])
        #     indices.append(idx)
        # elif grad < 0 and prev > 0 and len(buys) == len(sells):
        #     sells.append(data[idx + 1, 1])

        if abs(grad) > 1e-5:
            prev = grad

    if len(buys) > len(sells):
        sells.append(data[-1, 4])
        indices.append(len(data) - 1)

    profits = [sell - buy for buy, sell in zip(buys, sells)]
    return buys, sells, indices, profits, avg

In [ ]:
data = nikkei
window_size = 7
start = 1250
end = len(data)

data = data[start:end]
buys, sells, sell_indices, profits, avg = alg_moving_average(data, window_size=window_size)
accum_profits = list(accumulate(profits))
base_profits = data[:, 1] - data[0, 1]
plt.plot(sell_indices, accum_profits, label="algorithm") 
plt.plot(avg)
plt.plot(data[:, 1])
plt.plot(base_profits, label="baseline")
plt.legend()
plt.grid()

In [ ]:
profits